In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

In [ ]:
# Create configs to mount azure data lake
configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": "<client-id>",
    "fs.azure.account.oauth2.client.secret": "<client-secret>",
    "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/<tenant-id>/oauth2/token"
}

# Mount azure data lake to databricks
dbutils.fs.mount(
    source = "abfss://olympic-data-container@olympicdesa01.dfs.core.windows.net", # contrainer@storageacc
    mount_point = "/mnt/olympicdata",
    extra_configs = configs
)

Out[2]: True

In [ ]:
%fs

ls "/mnt/olympicdata"

path,name,size,modificationTime
dbfs:/mnt/olympicdata/processed/,processed/,0,1713772171000
dbfs:/mnt/olympicdata/raw/,raw/,0,1713768621000


In [ ]:
athletes = spark.read.format("csv").option("header", "true").load("/mnt/olympicdata/raw/athletes.csv")
coaches = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/olympicdata/raw/coaches.csv")
entriesgender = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/olympicdata/raw/gender_entries.csv")
medals = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/olympicdata/raw/medals.csv")
teams = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/olympicdata/raw/teams.csv")

In [ ]:
entriesgender = entriesgender.withColumn("Female",col("Female").cast(IntegerType()))\
    .withColumn("Male",col("Male").cast(IntegerType()))\
    .withColumn("Total",col("Total").cast(IntegerType()))

In [ ]:
type(entriesgender)

Out[15]: pyspark.sql.dataframe.DataFrame

In [ ]:
entriesgender.show()

+--------------------+------+----+-----+
|          Discipline|Female|Male|Total|
+--------------------+------+----+-----+
|      3x3 Basketball|    32|  32|   64|
|             Archery|    64|  64|  128|
| Artistic Gymnastics|    98|  98|  196|
|   Artistic Swimming|   105|   0|  105|
|           Athletics|   969|1072| 2041|
|           Badminton|    86|  87|  173|
|   Baseball/Softball|    90| 144|  234|
|          Basketball|   144| 144|  288|
|    Beach Volleyball|    48|  48|   96|
|              Boxing|   102| 187|  289|
|        Canoe Slalom|    41|  41|   82|
|        Canoe Sprint|   123| 126|  249|
|Cycling BMX Frees...|    10|   9|   19|
|  Cycling BMX Racing|    24|  24|   48|
|Cycling Mountain ...|    38|  38|   76|
|        Cycling Road|    70| 131|  201|
|       Cycling Track|    90|  99|  189|
|              Diving|    72|  71|  143|
|          Equestrian|    73| 125|  198|
|             Fencing|   107| 108|  215|
+--------------------+------+----+-----+
only showing top

In [ ]:
top_gold_medal_countries = medals.orderBy("Gold", ascending=False).select("Team_Country","Gold").show(5)

+--------------------+----+
|        Team_Country|Gold|
+--------------------+----+
|United States of ...|  39|
|People's Republic...|  38|
|               Japan|  27|
|       Great Britain|  22|
|                 ROC|  20|
+--------------------+----+
only showing top 5 rows



In [ ]:
average_entries_by_gender = entriesgender.withColumn(
    'Avg_Female', entriesgender['Female'] / entriesgender['Total']
).withColumn(
    'Avg_Male', entriesgender['Male'] / entriesgender['Total']
)
average_entries_by_gender.show()

+--------------------+------+----+-----+-------------------+-------------------+
|          Discipline|Female|Male|Total|         Avg_Female|           Avg_Male|
+--------------------+------+----+-----+-------------------+-------------------+
|      3x3 Basketball|    32|  32|   64|                0.5|                0.5|
|             Archery|    64|  64|  128|                0.5|                0.5|
| Artistic Gymnastics|    98|  98|  196|                0.5|                0.5|
|   Artistic Swimming|   105|   0|  105|                1.0|                0.0|
|           Athletics|   969|1072| 2041| 0.4747672709456149| 0.5252327290543851|
|           Badminton|    86|  87|  173|0.49710982658959535| 0.5028901734104047|
|   Baseball/Softball|    90| 144|  234|0.38461538461538464| 0.6153846153846154|
|          Basketball|   144| 144|  288|                0.5|                0.5|
|    Beach Volleyball|    48|  48|   96|                0.5|                0.5|
|              Boxing|   102

In [ ]:
athletes.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/olympicdata/processed/athletes")
coaches.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/olympicdata/processed/coaches")
entriesgender.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/olympicdata/processed/entriesgender")
medals.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/olympicdata/processed/medals")
teams.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/olympicdata/processed/teams")